In [21]:
import json
exp_json = '/Users/zacharymcgrath/Desktop/Experiment output/DEBUG OUTPUT/experiment_data.json'
save_file = '/Users/zacharymcgrath/Desktop/hybrid_pep_data.json'
exp = json.load(open(exp_json, 'r'))

peps = exp['experiment']

for p, pep in peps.items():
    if 'hybrid' in pep['analysis']['ranks']['correct_protein'].lower() and int(pep['analysis']['ranks']['sequence_length']) > 8:
        print(pep)
        json.dump(pep, open(save_file, 'w'))
        break


{'insulin': {'k=10': [0.0, 0.0, 0.16666666666666666, 0.16666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3333333333333333, 0.16666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.16666666666666666, 0.0, 0.0, 0.16666666666666666, 0.16666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.16666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'k=6': [0.0, 0.0, 0.25, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.25, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [23]:
parent_protein = 'HYBRID_0'
prots = exp['experiment_info']['proteins']

for p in prots:
    print(p)

{'name': 'insulin', 'sequence': 'MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQVGQVELGGGPGAGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN', 'identifier': 'id1'}
{'name': 'brain-derived-neurotrophic-factor', 'sequence': 'MTILFLTMVISYFGCMKAAPMKEANIRGQGGLAYPGVRTHGTLESVNGPKAGSRGLTSLADTFEHVIEELLDEDQKVRPNEENNKDADLYTSRVMLSSQVPLEPPLLFLLEEYKNYLDAANMSMRVRRHSDPARRGELSVCDSISEWVTAADKKTAVDMSGGTVTVLEKVPVSKGQLKQYFYETKCNPMGYTKEGCRGIDKRHWNSQCRTTQSYVRALTMDSKKRIGWRFIRIDTSCVCTLTIKRGR', 'identifier': 'id2'}
{'name': 'dixin', 'sequence': 'MLACLTRGNLLDVLQEGFNEQQLQAYVAWVNAQLKKRPAVKPVQDLRQDLRDGVILAYLIEIVAGEKLSGVQLSPGNQQEMKNNVEKVLQFVASKKIRMHQTSAKDIVDGNLKSIMRLVLALAAHFKPGSSRTVNQGRDSRAPLQSHRPHCATAVAQGAAAALADVCHDMSRSGRDVFRYRQRNSSMDEEIENPYWSVRALVQQYEGQQRSPSESSCSSLTSPSPIHSAKSESIITQSEEKADFVIIPAEGIENRTEGTDSPLSRDWRPGSPGTYLETSWEEQLLEQQEYLEKEMEEAKKMISGLQALLLNGSLPEDEQERPLALCEPGVNPEEQLIIIQSRLDQSMEENQDLKKELLKCKQEARNLQGIKDALQQRLTQQDTSVLQLKQELLRANMDKDELHNQNVDLQRKLDERNRLLGEYKKELGQKDRLLQQHQAKLEEALRKLSDVSYHQVDLERELEHKDVLLAHC